In [26]:
import pandas as pd

data = pd.read_csv('../../데이터/Transaction/transaction_final.csv',index_col=0)

In [27]:
def price_per_pyeong_interpolate(group):
    idx = pd.date_range(group['계약년월'].min(), group['계약년월'].max(), freq='MS')
    group = group.set_index('계약년월').reindex(idx)
    group['단지명'] = group['단지명'].fillna(method='ffill')
    group['시군구'] = group['시군구'].fillna(method='ffill')
    group['평단가'] = group['평단가'].interpolate()
    return group

In [51]:
from torch.utils.data import Dataset, DataLoader
import torch
import pandas as pd

class RNN_Dataset(Dataset):
    def __init__(self, data, sequence_length=5):
        interpolated_data = pd.DataFrame(data.groupby(['시군구', '단지명']).apply(price_per_pyeong_interpolate)['평단가']).reset_index().rename(columns={'level_2': '계약년월'})

        all_dongs_x, all_dongs_y = [], []

        for dong in interpolated_data['시군구'].unique():
            dong_x, dong_y = [], []
            for complex in interpolated_data[interpolated_data['시군구'] == dong]['단지명'].unique():
                complex_x, complex_y = [], []
                filtered_interpolated_data_values = interpolated_data[interpolated_data['단지명'] == complex]['평단가'].values
                for idx in range(len(filtered_interpolated_data_values) - sequence_length):
                    complex_x.append(torch.FloatTensor(filtered_interpolated_data_values[idx:idx + sequence_length]))
                    complex_y.append(torch.FloatTensor([filtered_interpolated_data_values[idx + sequence_length]]))
                dong_x.append(complex_x)
                dong_y.append(complex_y)
            all_dongs_x.append(dong_x)
            all_dongs_y.append(dong_y)

        self.x = all_dongs_x
        self.y = all_dongs_y

    def __getitem__(self, i):
        return self.x[i], self.y[i]

    def __len__(self):
        return len(self.x)


batch_size = 1
train_dataset = RNN_Dataset(data)
train_loader = DataLoader(train_dataset, batch_size=batch_size)

for x, y in train_loader:
    print(f"x의 shape: {x.shape}, y의 shape: {y.shape}")

TypeError: expected Tensor as element 0 in argument 0, but got list

In [8]:
import torch
from torch.utils.data import Dataset

class Filled_RNN_Dataset(Dataset):
    def __init__(self, data, sequence_length=5):
        interpolated_data = pd.DataFrame(data.groupby(['시군구','단지명']).apply(price_per_pyeong_interpolate)['평단가']).reset_index().rename(columns={'level_2': '계약년월'}).rename(columns={'level_2': '계약년월'})

        # max_complex_len = 0
        # for dong in interpolated_data['시군구'].unique():
        #     dong_unique_complexes_len = len(data[data['시군구']==dong]['단지명'].unique())
        #     if dong_unique_complexes_len > max_complex_len:
        #         max_complex_len = dong_unique_complexes_len

        dongs_X, dongs_Y = [], []

        for i in range(len(pd.to_datetime(pd.date_range(start="2006-01-01", end="2022-12-01", freq='MS')))-sequence_length):
            
        

        # dong_X, dong_Y = [], []
        # for complex in unique_complexes:
        #     filtered_data_values = data[data['단지명']==complex]['평단가'].values
        #     for i in range(len(filtered_data_values) - sequence_length):
        #         X.append(filtered_data_values[i:i+sequence_length])
        #         Y.append(filtered_data_values[i+sequence_length])
        # self.x = torch.FloatTensor(X)
        # self.y = torch.FloatTensor(Y)
        # self.len = len(X)

    def __getitem__(self, i):
        return self.x[i], self.y[i]

    def __len__(self):
        return self.len

sample_dataset = Filled_RNN_Dataset(data)

158
